# Interpolation Testbed Notebook

Short notebook to test various interpolation strategies. 

In [1]:
# Standard library
import sys
import os

# Data manipulation and analysis
import numpy as np
import pandas as pd
import xarray as xr

# Scientific computing and machine learning
from scipy.interpolate import griddata
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.preprocessing import MinMaxScaler

In [2]:
sys.path.insert(1, '../sealsml')

In [3]:
from baseline import gaussian_interp

## Loading in real data

Since none of these need to be 'trained', if it works on one dataset, it will work on all of them. 

In [4]:
data = '../data/training_data_SBL2m_Ug10_src10kg_b.0.nc'

In [5]:
ds = xr.open_dataset(data)
ds

<xarray.Dataset>
Dimensions:         (variable: 8, sample: 600, sensor: 10, time: 100, mask: 2,
                     pot_leak: 20, target_time: 1, sensor_loc: 3)
Coordinates:
  * variable        (variable) object 'ref_distance' 'ref_azi_sin' ... 'q_CH4'
  * sensor_loc      (sensor_loc) object 'xPos' 'yPos' 'zPos'
Dimensions without coordinates: sample, sensor, time, mask, pot_leak,
                                target_time
Data variables:
    encoder_input   (sample, sensor, time, variable, mask) float64 ...
    decoder_input   (sample, pot_leak, target_time, variable, mask) float64 ...
    target          (sample, pot_leak, target_time) float64 ...
    sensor_meta     (sample, sensor, sensor_loc) float64 ...
    leak_meta       (sample, pot_leak, sensor_loc) float64 ...
    met_sensor_loc  (sample, sensor_loc) float64 ...

## Dataset Exploration

This seems bad:

In [6]:
import warnings
from sklearn.exceptions import ConvergenceWarning

# Filter out ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

## I need to functionize this

In [7]:
dataset = gaussian_interp(data)
dataset

<xarray.Dataset>
Dimensions:           (SampleNumber: 600, MaxNumSensors: 10, MaxNumLeaks: 20,
                       x: 30, y: 30)
Coordinates:
  * SampleNumber      (SampleNumber) int64 0 1 2 3 4 5 ... 595 596 597 598 599
  * MaxNumSensors     (MaxNumSensors) int64 0 1 2 3 4 5 6 7 8 9
  * MaxNumLeaks       (MaxNumLeaks) int64 0 1 2 3 4 5 6 ... 13 14 15 16 17 18 19
  * x                 (x) float64 67.0 69.14 71.28 73.41 ... 124.7 126.9 129.0
  * y                 (y) float64 1.967e+03 1.969e+03 ... 2.027e+03 2.029e+03
Data variables:
    NumberOfSensors   (SampleNumber) int64 7 6 7 10 8 7 10 7 ... 8 6 5 7 9 7 7 8
    SensorMedian_ch4  (SampleNumber, MaxNumSensors) float64 3.152e-05 ... 0.0
    NumberOfLeaks     (SampleNumber) int64 12 19 18 16 15 15 ... 12 17 20 12 17
    LeakNumber        (SampleNumber) int64 5 0 14 0 1 4 0 3 ... 0 9 0 4 4 11 12
    PredLeakLocation  (SampleNumber, MaxNumLeaks) float64 0.0 0.0 ... 0.0 0.0
    Predch4value      (SampleNumber, MaxNumLeaks) float64 8.556e-06 ... 0.0
    interpolation     (SampleNumber, x, y) float64 3.523e-06 ... 1.034e-05